In [1]:
import numpy as np
import gym.spaces
from gridworld import GridworldEnv

In [2]:
env = GridworldEnv(shape=[4,4])
print('Number of states in the environment:', env.nS)
print('Number of actions in the environment:', env.nA)
print('\nEnv:')
env._render(mode='human', close=False)

Number of states in the environment: 16
Number of actions in the environment: 4

Env:
T  x  o  o
o  o  o  o
o  o  o  o
o  o  o  T


In [3]:
def iterate_value(env, threshold=1e-4, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    
    def one_step_lookahed(V, a, s):
        
        [(prob, next_state, reward, done)] = env.P[s][a]
        v = prob * (reward + discount_factor * V[next_state])
        
        return v
    
    # start with initial value function and initial policy
    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    
    # while not the optimal policy
    while True:
        
        # for stopping condition
        delta = 0
        
        # loop over state space
        for s in range(env.nS):
            
            actions_values = np.zeros(env.nA)
            
            # loop over possible actions
            for a in range(env.nA):
                
                # apply Bellman expectation equation to get action values
                actions_values[a] = one_step_lookahed(V, a, s)
                
            # pick the best action
            best_action_value = max(actions_values)
            
            # get the biggest difference between best action value and our old value function
            delta = max(delta, abs(best_action_value - V[s]))
            
            # apply Bellman optimality equation
            V[s] = best_action_value
            
            # to update the policy
            best_action = np.argmax(actions_values)
            
            # update the policy
            policy[s] = np.eye(env.nA)[best_action]
            
        # if optimal value function
        if (delta < threshold):
            break
            
    return policy, V

In [4]:
policy, V = iterate_value(env)

In [5]:
print('Policy Probability Distribution:')
print(policy)

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [6]:
print('Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):')
print(np.reshape(np.argmax(policy, axis=1), env.shape))

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]


In [7]:
print('Value Function:')
print(V)

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]


In [8]:
print('Reshaped Grid Value Function:')
print(V.reshape(env.shape))

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]


---